*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(train_file_path, header = 0, names = ['y', 'x'],  sep='\t', encoding = "utf-8")
test_dataset = pd.read_csv(test_file_path, header = 0, names = ['y', 'x'], sep='\t', encoding = "utf-8")

train_dataset.head()
train_dataset['x'].describe()

# Creating labesls
train_labels = train_dataset.pop('y')
test_labels = test_dataset.pop('y')


In [ ]:
# Cleaning data
import regex as re
def cleaningTweets(twt):
    twt = re.sub('@[A-ZÜÄÖa-züäöß0-9]+', '', twt)
    twt = re.sub('#', '', twt)
    twt = re.sub('https?:\/\/\+', '', twt)
    return twt

train_dataset.x = train_dataset.x.apply(cleaningTweets)
test_dataset.x = test_dataset.x.apply(cleaningTweets)

In [ ]:
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence

# Defining vocabulary and vocal size 
total_words = set()
for e in train_dataset.x.index:
  text = train_dataset.x[e]
  words = set(text_to_word_sequence(text))
  for w in words:
    if w not in total_words:
      total_words.add(w)



In [ ]:
for e in test_dataset.x.index:
  text = test_dataset.x[e]
  words = set(text_to_word_sequence(text))
  for w in words:
    if w not in total_words:
      total_words.add(w)


In [ ]:
VOCAB_SIZE = len(total_words)
print(VOCAB_SIZE)

In [ ]:
# Encoding strings into numbers using hashing_trick. I decide to use a word per word analysisn (I trided character per charachet but I doesn't complete the challenge!)
for e in train_dataset.x.index:
  train_dataset.x[e] = hashing_trick(train_dataset.x[e], round(vocab_size*1.3), hash_function='md5')

In [ ]:
for e in test_dataset.x.index:
  test_dataset.x[e] = hashing_trick(test_dataset.x[e], round(vocab_size*1.3), hash_function='md5')

In [ ]:
# Transforming labels from "ham"/"spam" into 0/1
train_labels = pd.factorize(train_labels)[0]
test_labels = pd.factorize(test_labels)[0]

In [ ]:
# Transforming dataframe into array
train_dataset.to_numpy()
test_dataset.to_numpy()

In [ ]:
train_dataset[0].size

In [ ]:
# homogenizing sizes

MAXLEN = 250
train_dataset = sequence.pad_sequences(train_dataset.x, MAXLEN)
test_dataset = sequence.pad_sequences(test_dataset.x, MAXLEN)


In [ ]:
# Creating the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, train_labels, epochs=10, validation_split=0.2)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  text_in_numbers = hashing_trick(pred_text, round(vocab_size*1.3), hash_function='md5')
  text_in_array = sequence.pad_sequences([text_in_numbers], MAXLEN)
  prediction = model.predict(text_in_array)
  if prediction[0] >= 0.5:
    result = [float(prediction[0]), 'spam']
  else:
    result = [float(prediction[0]), 'ham']
  return result

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
